In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

/home/jackson/vision-r1/vr1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_name = "/home/jackson/vision-r1/qwen-0.5-vl-custom"
max_pixels = 256 * 256
processor = AutoProcessor.from_pretrained(
    model_name, max_pixels=max_pixels, use_cache=False
)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",
    attn_implementation="flash_attention_2",
    use_cache=False,
)

In [87]:
processor = AutoProcessor.from_pretrained(model_name)

In [88]:
SYSTEM_PROMPT = """Respond in the following format:
<think>...</think>
<answer>...</answer>"""

In [89]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": SYSTEM_PROMPT}],
    },
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "/home/jackson/vision-r1/images/superCLEVR_new_000046.png",
            },
            {"type": "text", "text": "請計算裏面有多少物體"},
        ],
    },
]

text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda:0")

In [90]:
generated_ids = model.generate(**inputs, max_new_tokens=1024, do_sample=False)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


In [ ]:
print(output_text[0])